In [1]:
import graphlab as gl
test_data = gl.SFrame.read_csv("/Users/swastika.b/Documents/Swastika/SantaClaraEdu/courses/Machine Learning/Project/FinalCode/train.csv",verbose= False)
data = gl.SFrame.read_csv("/Users/swastika.b/Documents/Swastika/SantaClaraEdu/courses/Machine Learning/Project/FinalCode/training_data.csv",verbose= False)

This non-commercial license of GraphLab Create for academic use is assigned to sbhat1@scu.edu and will expire on August 12, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1503046013.log


# Preprocessing Data¶

In [2]:
def transform_text(text):
    
    import re
    import string
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    porter = PorterStemmer()
    
    lower_case_text = text.translate(None, string.punctuation).lower()
    words_list = word_tokenize(lower_case_text)
    text2 = []
    for word in words_list:
        text0 = word.decode('ascii', 'ignore')
        text1 = regex.sub(u'', text0)
        if not text1 == u'':
            if not text1 in stopwords.words('english'):
                text2.append(porter.stem(text1))
    return text2
    

data['comment_clean'] = data['Comment'].apply(transform_text)

In [3]:
def create_text(words):
    return ' '.join(words)

data['comment_string'] = data['comment_clean'].apply(create_text)

def remove_punctuation(text):
    import string
    lower_case_text = text.translate(None, string.punctuation).lower()
    return lower_case_text
    
#data['comment_string'] = data['Comment'].apply(remove_punctuation)

# Train the Data

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

def split_into_lemmas(comments):
    bigram_vectorizer = CountVectorizer(ngram_range=(1, 4), token_pattern=r'\b\w+\b', min_df=1)
    analyze = bigram_vectorizer.build_analyzer()
    return analyze(comments)

vectorizer = CountVectorizer(analyzer=split_into_lemmas,stop_words='english',strip_accents='ascii').fit(data['comment_string'])
text_transformed = vectorizer.transform(data['comment_string'])
tfidf_transformer = TfidfTransformer().fit(text_transformed)
tfidf_transformed_text = tfidf_transformer.transform(text_transformed)


In [12]:
from sklearn import linear_model
lr = linear_model.LogisticRegression(C=2)
lr.fit(tfidf_transformed_text,data['Insult'])


LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# K- Fold Cross Validation

In [13]:
from sklearn.cross_validation import cross_val_predict
from sklearn import metrics
import numpy as np
y = (data['Insult']).to_numpy()
predicted = cross_val_predict(lr, tfidf_transformed_text, y, cv=10)
metrics.accuracy_score(y, predicted) 

0.70693512304250561

# Evaluation on Test data

In [14]:
test_data['comment_clean'] = test_data['Comment'].apply(transform_text)
test_data['comment_string'] = test_data['comment_clean'].apply(create_text)
#test_data['comment_string'] = test_data['Comment'].apply(remove_punctuation)

In [15]:
test_text_transformed = vectorizer.transform(test_data['comment_string'])
tfidf_transformed_test_text = tfidf_transformer.transform(test_text_transformed)
predicted_class = lr.predict(tfidf_transformed_test_text)

In [16]:
import numpy as np
true_class = np.array(test_data['Insult'])
from sklearn.metrics import accuracy_score
accuracy_score(true_class, predicted_class)

0.70230554851786164

In [10]:
len(data)

2235